<a href="https://colab.research.google.com/github/kykytsapol/-YouTube-/blob/main/%D0%98%D1%82%D0%BE%D0%B3_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%B8%D0%BA%D0%B0_%D0%B8%D0%B7%D0%BC%D0%B5%D1%80%D0%B5%D0%BD%D0%B8%D1%8F_%D1%80%D0%B5%D0%B0%D0%BA%D1%86%D0%B8%D0%B8_%D0%B0%D1%83%D0%B4%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B8_%D0%BD%D0%B0_YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# для измерении реакции аудтории нам понадобятся следующие библиотеки:
import pandas as pandas
#from randan.descriptive_statistics import ScaleStatistics
import numpy as np
import scipy.stats
import math
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import gensim
from gensim.models import LdaModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB

### Анализ числовых метрик

In [ ]:
# Открываем файл с найденными каналами и их метриками
channels = pandas.read_excel('ваши_каналы.xlsx', index_col = 0)

In [ ]:
# Открываем файл с найденными видео и их метриками
videos = pandas.read_excel('ваши_видео.xlsx', index_col = 0)

In [ ]:
# Выводим описательные статистики для метрик канала: количество подписчиков и видео
ss = ScaleStatistics(channels, ['statistics.subscriberCount', 'statistics.videoCount'], normality_test=True)

In [ ]:
# Выводим описательные статситики для метрик видео: количество просмотров, лайков и комментриев
ss = ScaleStatistics(videos, ['statistics.viewCount', 'statistics.likeCount', 'statistics.commentCount'], normality_test=True)

In [ ]:
# Рассчитываем вовлеченность аудитории
videos['invl']= videos['statistics.likeCount']\
                + videos['statistics.commentCount']

videos[['invl',
      'statistics.likeCount',
      'statistics.viewCount']]

In [ ]:
# Выводим описательные статистики для вовлеченности аудитории
ss = ScaleStatistics(videos, ['invl'], normality_test=True)

In [ ]:
# Эта функция позволяет рассчитать доверительный интервал
# С помощью этой функции вы можете поситать доверительный интервал для среднего или медианного значения
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a) # мы строили для медианы, так как у нас было смещение в данных. Можно поменять на mean (ср. знач.)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
# Расчет доверительного интервала для количества подписчиков (вероятность 95%)
print(mean_confidence_interval(channels['statistics.subscriberCount'], confidence=0.95))

In [ ]:
# Расчет доверительного интервала для количества видео (вероятность 95%)
print(mean_confidence_interval(channels['statistics.videoCount'], confidence=0.95))

In [ ]:
# Расчет доверительного интервала для количества просмотров (вероятность 95%)
print(mean_confidence_interval(videos['statistics.viewCount'], confidence=0.95))

In [ ]:
# Расчет доверительного интервала для количества комментриев (вероятность 95%)
print(mean_confidence_interval(videos['statistics.commentCount'], confidence=0.95))

In [ ]:
# Расчет доверительного интервала для количества лайков (вероятность 95%)
print(mean_confidence_interval(videos['statistics.likeCount'], confidence=0.95))

In [ ]:
# Расчет доверительного интервала для вовлеченности (вероятность 95%)
print(mean_confidence_interval(videos['invl'], confidence=0.95))

Данный алгорит анализа метрик каналов и видео вы можете проделать несколько раз, если вам неоходимо изучть динамику. С помощь анализа доверительных интервалов вы можете определить, значимо ли различаются изучаемые метрики каналов (в двух периодах).

### Тематическое моделирование

In [ ]:
# открываем экселевский файл с комментриями
comments = pandas.read_excel('ваш_файл_с_комментариями.xlsx', index_col = 0 )
comments

In [ ]:
# Концертируем наш датафрейм в лист
data = comments['столбец_с_преодобработанным_текстом'].values.tolist()

In [ ]:
# Преобразование документа в список токенов 
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence))) 
data_words = list(sent_to_words(data))

In [ ]:
# Создаем словарь
id2word = corpora.Dictionary(data_words)

In [ ]:
# Создаем корпус
texts = data_words

In [ ]:
# Частота встречаемости 
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
# эта функция ползволит определить сколько топиков необходимо добавить в модель, чтобы достишь наивысшего значения согласованности
def compute_coherence_values(dictionary,corpus,texts,limit,start=2,step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaModel(corpus=corpus, num_topics=num_topics,id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [ ]:
# задаем наши значения
model_list, coherence_values = compute_coherence_values(dictionary=id2word,corpus=corpus,
                                                      texts=texts, start=2, limit=50, step=5)

In [ ]:
# Сторим график, который показывает, как изменяется значение согласованности при добавлении топика в модель.
limit=50; start=2; step=5;
x= range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Кол-во топиков")
plt.ylabel("Согласованность")
plt.legend(("Изменение согласованности"), loc='best')
plt.show()

In [ ]:
# Строим LDA модель 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 5) # указываем нужное количество топиков

In [ ]:
# Можно вывести точное значение согласованности модели, рассчитанное методом c_v 
cm = CoherenceModel(model=lda_model, corpus=corpus, texts=texts,
                              dictionary=id2word, coherence='c_v')
coherence = cm.get_coherence()
coherence

In [ ]:
# Можно вывести точное значение согласованности модели, рассчитанное методом u_mass 
cm = CoherenceModel(model=lda_model, corpus=corpus,
                              dictionary=id2word, coherence='u_mass')
coherence = cm.get_coherence()
coherence

In [ ]:
# Выводим получишвшиеся результаты: топики и их ключевые слова с весами 
lda_model.print_topics()

In [ ]:
# Создаем датафрейм с оргинальным текстом, чтобы добавить его к таблице ниже 
comments.dropna()
comments.index = range(0, len(comments))
comments_list = comments['столбец_с_оригинальным_текстом'].values.tolist()

In [ ]:
#Тут мы определяем какой документ к какой теме преимущественно относится
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=comments_list):
    sent_topics_df = pandas.DataFrame()
    # Получаем каждый топик для каждого документа 
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Получите доминирующую тему, процентный вклад и ключевые слова для каждого документа
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => доминирующий топик 
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pandas.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Доминирующий_топик', 'Процентный_вклад', 'Ключевые_слова']
    # Add original text to the end of the output
    contents = pandas.Series(texts)
    sent_topics_df = pandas.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=comments_list)
# формат
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Номер_документа', 'Доминирующий_топик', 'Процентный_вклад', 'Ключевые_слова', 'Оригинальный_текст']
# смотрим
df_dominant_topic.head(10)

In [ ]:
# Смотрим сколько документов попало в каждую тему
df_dominant_topic['Доминирующий_топик'].value_counts()

In [ ]:
# Добавим еще к таблице предобработанный текст 
df_dominant_topic['Преобразованный_текст'] = comments['столбец_с_преодобработанным_текстом']

In [ ]:
df_dominant_topic

In [ ]:
# Для интерпретации можно также вывести наиболее связанные с топиками комментарии
pandas.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pandas.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Доминирующий_топик')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pandas.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Процентный_вклад'], ascending=False).head(1)], 
                                            axis=0)
  
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

sent_topics_sorteddf_mallet.columns = ['Номер_документа', "Процентный_вклад", "Ключевые_слова", "Оригинальный_текст"]

sent_topics_sorteddf_mallet

Если вы изучаете разные временные периоды, то вы можете с помощью данного кода построить несколько моделей. Скопируйте код и поменяйте файл с данными.

### Сентимент анализ

In [ ]:
#Открываем размеченный датафрейм (с тональной отметкой)
df = pandas.read_excel("ваш_размеченный_файл.xlsx",index_col=0)
df.head()

Датафрейм, который мы использовавли вы можете скачать по ссылке: http://study.mokoron.com/#download 

In [ ]:
# Задаем x и y
X = df['text_clear']
y = df['label']

In [ ]:
# Делим на тестовую и обучающу выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
# Настраиваем класс CountVectorizer
vectorizer = CountVectorizer(min_df=2, max_features=10000)
# Далее подаем в метод transform класса CountVectorizer обучающую выборку
train_vectors = vectorizer.fit_transform(X_train.values.astype('U'))
test_vectors = vectorizer.transform(X_test.values.astype('U'))

In [ ]:
# Вам потребуется примерно 3 минут, чтобы обучить модель. Оптимальные гиперпараметры
model = BernoulliNB(alpha=10.0)
model.fit(train_vectors, y_train)

In [ ]:
text = vectorizer.transform(df_dominant_topic["Преобразованный_текст"].values.astype('U'))

In [ ]:
# Прдесказываем тональность для наших данных
sentiment_text = model.predict(text)

In [ ]:
df_dominant_topic['sentiment_text'] = sentiment_text

In [ ]:
# Смотрим сколько всего негавтивных и позитивных текстов.
df_dominant_topic.groupby('sentiment_text').sentiment_text.count()

In [ ]:
# Доверительный интервал для доли позитивных и негативных комментриев
neg = # число негативных комментариев
pos = # число позитивных комментариев
total = # общее число комментариев
a = 1.96 # с вероятностью 95%, по желанию можно поменять значение z
pos_proc = pos/total # процент позитивных комментариев
neg_proc = neg/total # процент негативных комментариев
s = math.sqrt(pos_proc * neg_proc)  # оценка стандартного отклонения
s_doli = s/(math.sqrt(total)) # стандартная ошибка  доли

# Верхнии и нижнии границы доверительных интервалов 
nig_gran_pos = pos_proc - a * s_doli
verch_gran_pos = pos_proc + a * s_doli

nig_gran_neg = neg_proc - a * s_doli
verch_gran_neg = neg_proc + a * s_doli
print('Для доли положительных комментариев:', nig_gran_pos, verch_gran_pos, \
      'Для доли негативных комментариев:', nig_gran_neg, verch_gran_neg)

In [ ]:
df_dominant_topic.groupby(["Доминирующий_топик", 'sentiment_text']).Доминирующий_топик.aggregate('count')

In [ ]:
df_dominant_topic.groupby(["Доминирующий_топик"]).sentiment_text.aggregate('mean')

Если вы изучаете разные временные периоды, то вы можете с помощью данного кода построить несколько моделей. Скопируйте код и поменяйте файл с данными.